# Regresión Logística

## Paso 1: Importar librerías

In [101]:
##Cargue de las librerías
import pandas as pd
import numpy as np
##Librerías para graficar
import matplotlib.pyplot as plt
import seaborn as sns
##Librerías para test de normalidad
import statsmodels.api as sm
from scipy import stats
##Librerias para preprocesamiento
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
##Librerías para over y under sampling
from sklearn.utils import resample

## Paso 2: Funciones

In [119]:
# Función para importar data .xlsx o .csv
def importDatos(ruta, archivo, ext):
    if ext == '.csv':
        data = pd.read_csv(ruta+archivo+ext)
        return data
    elif ext == '.xlsx':
        data = pd.read_excel(ruta+archivo+ext)
        return data
    else:
        print('Extensión diferente')

# Función informe sobre los datos NAN


def datos_NA(df):
    if isinstance(df, pd.DataFrame):
        total_na = df.isna().sum().sum()
        print("Dimensiones : %d filas, %d columnas" %
              (df.shape[0], df.shape[1]))
        print("Total Valores NA : %d " % (total_na))
        print("%38s %10s     %10s %10s" %
              ("Nombre Columna", "Tipo de Dato", "#Distintos", "Valores NA"))
        col_name = df.columns
        dtyp = df.dtypes
        uniq = df.nunique()
        na_val = df.isna().sum()
        for i in range(len(df.columns)):
            print("%38s %10s   %10s %10s" %
                  (col_name[i], dtyp[i], uniq[i], na_val[i]))

    else:
        print("Se esperaba dataframe %15s" % (type(df)))

# Función para pintar algo el histograma reemplazamos con la media o mediana


def histoPlot(df, nomCol, anchoBar):
    sns.histplot(data=df, x=nomCol, binwidth=anchoBar, kde=True)

# Función para reemplazo de valore numéricos con la mediana


def imputacionNa(df, nomCol, param):
    if param == 'media':
        df[nomCol] = df[nomCol].fillna(df[nomCol].mean())
        return df[nomCol]
    else:
        df[nomCol] = df[nomCol].fillna(df[nomCol].median())
        return df[nomCol]
# Funcion para revisar datos categóricos o texto


def revCategoricos(df, nomCol):
    rev = df[nomCol].value_counts()
    return rev

# Función para datos categóricos modificaciones


def imputacionCat(df, nomCol, busqueda, reempl):
    df[nomCol] = np.where(df[nomCol] == busqueda, reempl, df[nomCol])
    return df[nomCol]

# Función para reeemplazo NA categóricos


def imputacionCatNa(df, nomCol):
    df[nomCol] = df[nomCol].fillna(df[nomCol].mode()[0])
    return df[nomCol]

# Función para cambiar datos de texto a numerico float64


def cambioTextNum(df, nomCol, tipo):
    df[nomCol] = df[nomCol].astype(tipo)
    return df[nomCol]

# Función modificar a 1 y 0 los datos categóricos


def camTextBi(df, nomCol, busqueda, camuno, camdos):
    df[nomCol] = np.where(df[nomCol] == busqueda, camuno, camdos)
    return df[nomCol]

# Función para variables dummies


def convDummies(df, nomCol):
    dummies = pd.get_dummies(df[nomCol])
    data = pd.concat([df, dummies], axis=1)
    data.drop(columns=[nomCol], inplace=True)
    return data

# Función cambiar encabezados del dataframe


def camEncab(df, listaN):
    df.columns = listaN
    return df

# Función para el test de Shapiro - wilk - resistente al ruido en los datos


def testShapiroWilk(df):
    valoresP = []
    concepto = []
    variable = []
    for column in df:
        k2, p_value = stats.shapiro(df[column].values)
        valoresP.append(p_value)
        variable.append(column)
        if (p_value < 0.05):
            concepto.append('No es una variable Normal')
        else:
            concepto.append('Es una variable Normal')
    dfShapiro = pd.DataFrame(
        {'Variable': variable, 'Valores P': valoresP, 'Concepto': concepto})
    return dfShapiro

# Función para normalizar la data min - max sklearn


def normaData(df):
    valores = df.values  
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(valores)

    pd.DataFrame(np.vstack((scaler.data_min_, scaler.data_max_)),
             index=['Min', 'Max'],
             columns=df.columns)

    normalizados = scaler.transform(valores)
    df_norm = pd.DataFrame(normalizados,
                       index=df.index,
                       columns=df.columns)
    return df_norm

## Función para Estandarizar la data formula z-score
def stanData(df):
    scaler = StandardScaler()
    standardized_data = scaler.fit_transform(df)
    df_stan = pd.DataFrame(standardized_data, columns=df.columns)
    return df_stan

## Función para hacer la matriz de correlación de manera Tidy
def tidy_corr_matrix(corr_mat):
    '''
    Función para convertir una matriz de correlación de pandas en formato tidy.
    '''
    corr_mat = corr_mat.stack().reset_index()
    corr_mat.columns = ['variable_1','variable_2','r']
    corr_mat = corr_mat.loc[corr_mat['variable_1'] != corr_mat['variable_2'], :]
    corr_mat['abs_r'] = np.abs(corr_mat['r'])
    corr_mat = corr_mat.sort_values('abs_r', ascending=False)
    
    return(corr_mat)

##Función de label encoder
def Codificacion(df, nomCol):
    label_e = LabelEncoder()
    df[nomCol] = label_e.fit_transform(df[nomCol])
    return df[nomCol]



## Paso 3: Llamado a Funciones

In [103]:
X = importDatos("/home/julian/Documentos/DiplomadoML/DocumentacionDiplomadoML/4. Aprendizaje Supervisado Clasificación/Datasets/", 'X','.csv')
y = importDatos("/home/julian/Documentos/DiplomadoML/DocumentacionDiplomadoML/4. Aprendizaje Supervisado Clasificación/Datasets/", 'y','.csv')


In [104]:
##Revisión de datos NAN
datos_NA(X)

Dimensiones : 36881 filas, 24 columnas
Total Valores NA : 74668 
                        Nombre Columna Tipo de Dato     #Distintos Valores NA
                              Location     object           46          0
                               MinTemp    float64          369        338
                               MaxTemp    float64          470        242
                              Rainfall    float64          432        626
                           Evaporation    float64          215      12846
                              Sunshine    float64          144      13564
                           WindGustDir     object           16       3368
                         WindGustSpeed    float64           64       3361
                            WindDir9am     object           16       2809
                            WindDir3pm     object           16        962
                          WindSpeed9am    float64           39        662
                          WindSpeed3pm    f

In [105]:
##Valores NAN
datos_NA(y)

Dimensiones : 36881 filas, 1 columnas
Total Valores NA : 620 
                        Nombre Columna Tipo de Dato     #Distintos Valores NA
                          RainTomorrow     object            2        620


In [106]:
###Limpieza de datos NAN
revCategoricos(X, 'Location')

##Crear variables dummies para cada Location hacer a lo ultimo
dataFinal = convDummies(X, 'Location')

In [107]:
##Limpiar NAN mintemp y max temp
X['MinTemp']= imputacionNa(X, 'MinTemp','Mediana')
X['MaxTemp']= imputacionNa(X, 'MaxTemp','Mediana')
X['Rainfall']= imputacionNa(X, 'Rainfall','Mediana')
X['Evaporation']= imputacionNa(X, 'Evaporation','Mediana')
X['Sunshine']= imputacionNa(X, 'Sunshine','Mediana')
X['WindGustSpeed']= imputacionNa(X, 'WindGustSpeed','Mediana')

X['WindSpeed9am']= imputacionNa(X, 'WindSpeed9am','Mediana')
X['WindSpeed3pm']= imputacionNa(X, 'WindSpeed3pm','Mediana')
X['Humidity9am']= imputacionNa(X, 'Humidity9am','Mediana')
X['Humidity3pm']= imputacionNa(X, 'Humidity3pm','Mediana')
X['Pressure9am']= imputacionNa(X, 'Pressure9am','Mediana')
X['Pressure3pm']= imputacionNa(X, 'Pressure3pm','Mediana')
X['Cloud9am']= imputacionNa(X, 'Cloud9am','Mediana')
X['Cloud3pm']= imputacionNa(X, 'Cloud3pm','Mediana')
X['Temp9am']= imputacionNa(X, 'Temp9am','Mediana')
X['Temp3pm']= imputacionNa(X, 'Temp3pm','Mediana')



In [108]:
##Limpieza Categóricos
X['WindGustDir']= imputacionCatNa(X, 'WindGustDir')
X['WindDir9am']= imputacionCatNa(X, 'WindDir9am')
X['WindDir3pm']= imputacionCatNa(X, 'WindDir3pm')
X['RainToday']= imputacionCatNa(X, 'RainToday')

In [109]:
##Revisión general
datos_NA(X)

Dimensiones : 36881 filas, 24 columnas
Total Valores NA : 0 
                        Nombre Columna Tipo de Dato     #Distintos Valores NA
                              Location     object           46          0
                               MinTemp    float64          369          0
                               MaxTemp    float64          470          0
                              Rainfall    float64          432          0
                           Evaporation    float64          215          0
                              Sunshine    float64          144          0
                           WindGustDir     object           16          0
                         WindGustSpeed    float64           64          0
                            WindDir9am     object           16          0
                            WindDir3pm     object           16          0
                          WindSpeed9am    float64           39          0
                          WindSpeed3pm    float

In [110]:
##Convertir a Número variables categóricas
dataN = convDummies(X, 'Location')

In [111]:
dataN.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,...,SydneyAirport,Townsville,Tuggeranong,WaggaWagga,Walpole,Watsonia,Williamtown,Witchcliffe,Wollongong,Woomera
0,5.1,14.3,0.0,1.8,8.9,NW,30.0,WSW,SSE,7.0,...,0,0,0,0,0,0,0,0,0,0
1,1.1,14.5,0.4,4.6,8.4,SSW,50.0,E,SW,7.0,...,0,0,0,0,0,0,0,0,0,0
2,19.7,26.2,0.0,7.2,7.2,SSE,41.0,SSE,SSE,19.0,...,0,0,0,0,0,0,1,0,0,0
3,16.6,28.0,0.0,9.0,11.3,SW,54.0,SSE,SW,28.0,...,0,0,0,0,0,0,0,0,0,0
4,14.6,25.3,0.0,4.6,8.4,NNW,43.0,WNW,N,6.0,...,0,0,0,0,0,0,0,0,0,0


In [112]:
revCategoricos(X, 'WindGustDir')

SE     5771
W      2375
SSE    2330
N      2328
S      2317
SW     2272
E      2209
WSW    2206
SSW    2139
WNW    2052
ENE    2026
NW     1940
ESE    1894
NE     1769
NNE    1663
NNW    1590
Name: WindGustDir, dtype: int64

In [118]:
##Cambiar a North, South, East, West, NW, NE, SE, SW - label encoder
X.WindGustDir = imputacionCat(X, 'WindGustDir', 'SSE','SE')
X.WindGustDir = imputacionCat(X, 'WindGustDir', 'WSW','SW')
X.WindGustDir = imputacionCat(X, 'WindGustDir', 'SSW','SW')
X.WindGustDir = imputacionCat(X, 'WindGustDir', 'WNW','NW')
X.WindGustDir = imputacionCat(X, 'WindGustDir', 'ENE','NE')
X.WindGustDir = imputacionCat(X, 'WindGustDir', 'ESE','SE')
X.WindGustDir = imputacionCat(X, 'WindGustDir', 'NNE','NE')
X.WindGustDir = imputacionCat(X, 'WindGustDir', 'NNW','NW')

##Label encoder
from sklearn import preprocessing

label_e = preprocessing.LabelEncoder()

X.WindGustDir = label_e.fit_transform(X.WindGustDir)

revCategoricos(X, 'WindGustDir')

5    9995
6    6617
3    5582
2    5458
7    2375
1    2328
4    2317
0    2209
Name: WindGustDir, dtype: int64

In [121]:
revCategoricos(X, 'WindDir9am')

5    6586
2    6090
3    5954
6    5754
1    5671
0    2375
4    2233
7    2218
Name: WindDir9am, dtype: int64

In [120]:
##Cambiar a North, South, East, West, NW, NE, SE, SW 
X.WindDir9am = imputacionCat(X, 'WindDir9am', 'SSE','SE')
X.WindDir9am = imputacionCat(X, 'WindDir9am', 'WSW','SW')
X.WindDir9am = imputacionCat(X, 'WindDir9am', 'SSW','SW')
X.WindDir9am = imputacionCat(X, 'WindDir9am', 'WNW','NW')
X.WindDir9am = imputacionCat(X, 'WindDir9am', 'ENE','NE')
X.WindDir9am = imputacionCat(X, 'WindDir9am', 'ESE','SE')
X.WindDir9am = imputacionCat(X, 'WindDir9am', 'NNE','NE')
X.WindDir9am = imputacionCat(X, 'WindDir9am', 'NNW','NW')

##Label encoder
X.WindDir9am = Codificacion(X, 'WindDir9am')

In [123]:
revCategoricos(X, 'WindDir3pm')

5    8445
6    6781
3    6246
2    5846
4    2547
7    2486
1    2278
0    2252
Name: WindDir3pm, dtype: int64

In [122]:
##Cambiar a North, South, East, West, NW, NE, SE, SW 

X.WindDir3pm = imputacionCat(X, 'WindDir3pm', 'SSE', 'SE') 
X.WindDir3pm = imputacionCat(X, 'WindDir3pm', 'WSW', 'SW')
X.WindDir3pm = imputacionCat(X, 'WindDir3pm', 'SSW', 'SW')
X.WindDir3pm = imputacionCat(X, 'WindDir3pm', 'WNW', 'NW')
X.WindDir3pm = imputacionCat(X, 'WindDir3pm', 'ENE', 'NE')
X.WindDir3pm = imputacionCat(X, 'WindDir3pm', 'ESE', 'SE')
X.WindDir3pm = imputacionCat(X, 'WindDir3pm', 'NNE', 'NE')
X.WindDir3pm = imputacionCat(X, 'WindDir3pm', 'NNW', 'NW')

X.WindDir3pm = Codificacion(X, 'WindDir3pm')

In [124]:
revCategoricos(X, 'RainToday')

No     28382
Yes     8499
Name: RainToday, dtype: int64

In [125]:
##Binarizar No =0 y Si=1
X['RainToday'] = camTextBi(X, 'RainToday', 'No', 0, 1)
revCategoricos(X, 'RainToday')

0    28382
1     8499
Name: RainToday, dtype: int64

In [126]:
##Imputación de variables predictora
y['RainTomorrow'] = imputacionCatNa(y, 'RainTomorrow')
y['RainTomorrow'] = camTextBi(y, 'RainTomorrow', 'No', 0, 1)


In [128]:
revCategoricos(y, 'RainTomorrow')

0    28352
1     8529
Name: RainTomorrow, dtype: int64

In [127]:
datos_NA(y)

Dimensiones : 36881 filas, 1 columnas
Total Valores NA : 0 
                        Nombre Columna Tipo de Dato     #Distintos Valores NA
                          RainTomorrow      int64            2          0
